In [1]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [2]:
#登入pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [3]:
#確認裡面有哪幾個database
clinet.database_names()

['admin', 'iii-2017-07', 'local', 'mydb', 'rawData', 'test', 'test-database']

In [4]:
#選擇rawData這個db
db=clinet['rawData']

In [5]:
#確認裡面有哪幾個collection
db.collection_names()

['mobile01_tainan',
 'TainanUnique',
 'merge_tainanUni',
 'ptt_tainan_address',
 'AipinwanFood',
 'pixnet_tainan',
 'ptt_tainan_tel',
 'pixnet_tainan_author',
 'ptt_tainan']

In [6]:
#將愛評網讀入
ipeenjson=list(db.TainanUnique.find({}))

In [33]:
pinlist

['食物美味',
 '有點好吃',
 '好吃',
 '很好吃',
 '非常好吃',
 '美味',
 '很美味',
 '非常美味',
 '好好吃',
 '好食',
 '超好吃',
 '超愛',
 '非吃不可',
 'hen好吃',
 '過癮',
 '開胃',
 '食指大動',
 '甘美多汁',
 '垂涎三尺',
 '有果味',
 '柔軟',
 '酥脆',
 '有彈性',
 '很嫩',
 '特推',
 '好愛吃',
 '很脆',
 '飯超香',
 '不難吃',
 '超優',
 '我愛它',
 '好扎實',
 '愛這道',
 '還不錯',
 '還不賴',
 '不卡牙',
 '鮮甜',
 '愛慘',
 '挺愛的',
 '對胃口',
 '耐吃',
 '有扎實',
 '好喝',
 '這必點',
 '愛它',
 '合胃口',
 '跌破',
 '忘不掉',
 '驚艷感',
 '刮目相看',
 '鮮嫩',
 '不會過於甜膩',
 '不會過甜',
 '不會過膩',
 '綿綿的',
 '熱呼呼的',
 '軟嫩',
 'Q彈',
 '重質',
 '不膩口',
 '蠻不錯',
 '爽口',
 '清爽',
 '蠢蠢欲動',
 '剛好',
 '有層次',
 '刺激味蕾',
 '出色',
 '解渴',
 '很清爽',
 '沒有羊騷味',
 '好入味',
 '不油膩',
 '一品',
 '也不錯',
 '相當軟爛',
 '入口即化',
 '超級愛',
 '無法忘懷',
 '甜而不膩',
 '非常厲害',
 '讚不絕口',
 '薄可透光',
 '有別於一般',
 '綿密口感',
 '令人驚豔',
 '這麼好吃',
 '多層次',
 '甘願排隊',
 '屹立不搖',
 '誘人',
 '否食物美味',
 '味道較不好',
 '難吃',
 '很難吃',
 '難ㄘ',
 '頗難吃',
 '不太好吃',
 '有點普通',
 '覺得普普',
 '非常難吃',
 '非常不好吃',
 '沒味道',
 '超難吃',
 '到膩',
 '沒肉',
 '不對味',
 '不太愛吃',
 '不會再來',
 '雷',
 '很雷',
 '燒焦',
 '噁心',
 '油膩',
 '淡而無味',
 '黏糊糊',
 '咬不動',
 '不新鮮',
 '不愛吃',
 '油了點',
 '差強人意',
 '太濕軟',
 '大失所望',
 '不愛它

# 將結巴字典加入自己的許多詞

In [7]:

#結巴 匯入自己的詞典
jieba.set_dictionary('D:/WordLibrary/JiebaUse/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("D:\Data\csvtest\MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]

#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
    

Building prefix dict from D:\WordLibrary\JiebaUse\jieba_dict.txt.big ...
Loading model from cache C:\Users\Java\AppData\Local\Temp\jieba.uf3aabe1eaf7c86dda8313189b0c5c683.cache
Loading model cost 1.022 seconds.
Prefix dict has been built succesfully.


In [38]:
wordlist

{'',
 '青綠色',
 '烤雞',
 '甲頂',
 '碗盆',
 '浸漬',
 '嚥',
 '禁菸',
 '糊塗',
 '一時衝動',
 '風所',
 '他的紙',
 '還撒',
 '青蔥',
 '飽食',
 '盜用',
 '一回',
 '蔓生',
 '穿到',
 '柳下屋',
 '街剛',
 '好心',
 '逛下來',
 '多不熟',
 '台助',
 '以盤',
 '圍繞',
 '身旁',
 '一下週',
 '夥計',
 '結尾',
 '溢發',
 '選熟度',
 '代購',
 '和文',
 '三布',
 '補習班',
 '沒請',
 '覬覦',
 '般的',
 '公賣',
 '人煙稀少',
 '完底',
 '十餘個',
 '亞麻子',
 '第十二',
 '本周',
 '小烤',
 '暢通',
 '眼見',
 '七分鐘',
 '元大',
 '育',
 '分甜',
 '歸還',
 '起冰來',
 '這碗湯',
 '服務項目',
 '撲人',
 '愛又恨',
 '酒醬',
 '量詞',
 '超驚艷',
 '四段',
 '以涼',
 '弄髒了',
 '屋官網',
 '屋台',
 '烤山',
 '雕像',
 '事要',
 '湯底',
 '緣份',
 '第三回',
 '利落',
 '會化',
 '壯大',
 '百貨業',
 '黃水晶',
 '柳營',
 '視食',
 '戰火',
 '梅鳳飲食店',
 '一姐',
 '各憑本事',
 '麻雀',
 '樓上',
 '抵擋',
 '伴你咖啡',
 '這種',
 '臻',
 '極佳',
 '元裡面',
 '推來推去',
 '膽',
 '遊園',
 '炆',
 '木納',
 '黃豆',
 '服務站',
 '來板',
 '累累',
 '兩店',
 '演譯',
 '合',
 '斯哩馬謝',
 '迅勢',
 '岩本',
 '海龜',
 '身畫',
 '需在',
 '林豆',
 '莧菜',
 '北成',
 '街間',
 '名東',
 '蕭條',
 '荀',
 '以淡',
 '場',
 '風茹',
 '多辣',
 '全制',
 '雪漾冰室',
 '一株',
 '像樣',
 '小澀',
 '紅白',
 '剪報',
 '秒鐘',
 '影月點',
 '甜到',
 '聞茶',
 '豬8色韓式料理',
 '著熱',
 '鍋為',
 '擠壓',
 '拔

# 將許多可能會用來分析的項目做成list

# 做TD-IDF分析

In [8]:

st=time.time()
#取得各店家的斷完詞評論contentcutlist
contentcutlist = []
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
cutcount=0
for contentcombine in commentcombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(contentcombine, cut_all=False)
    contentcut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    contentcutlist.append(contentcut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(contentcut.split(" ")).most_common(300) for contentcut in contentcutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for contentcut in contentcutlist:
        if word in contentcut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

以切200篇文章的詞
以切400篇文章的詞
以切600篇文章的詞
以切800篇文章的詞
以切1000篇文章的詞
以切1200篇文章的詞
以切1400篇文章的詞
以切1600篇文章的詞
以切1800篇文章的詞
以切2000篇文章的詞
以切2200篇文章的詞
以切2400篇文章的詞
以切2600篇文章的詞
以切2800篇文章的詞
以切3000篇文章的詞
以切3200篇文章的詞
以切3400篇文章的詞
以切3600篇文章的詞
以切3800篇文章的詞
以切4000篇文章的詞
以切4200篇文章的詞
以切4400篇文章的詞
以切4600篇文章的詞
以切4800篇文章的詞
以切5000篇文章的詞
以切5200篇文章的詞
以切5400篇文章的詞
以切5600篇文章的詞
以計算1000個文字的出現篇數
以計算2000個文字的出現篇數
以計算3000個文字的出現篇數
以計算4000個文字的出現篇數
以計算5000個文字的出現篇數
以計算6000個文字的出現篇數
以計算7000個文字的出現篇數
以計算8000個文字的出現篇數
以計算9000個文字的出現篇數
以計算10000個文字的出現篇數
以計算11000個文字的出現篇數
以計算12000個文字的出現篇數
以計算13000個文字的出現篇數
以計算14000個文字的出現篇數
以計算15000個文字的出現篇數
以計算16000個文字的出現篇數
以計算17000個文字的出現篇數
以計算18000個文字的出現篇數
以計算19000個文字的出現篇數
以計算20000個文字的出現篇數
以計算21000個文字的出現篇數
以計算22000個文字的出現篇數
以計算23000個文字的出現篇數
以計算24000個文字的出現篇數
以計算25000個文字的出現篇數
以計算26000個文字的出現篇數
以計算27000個文字的出現篇數
以計算28000個文字的出現篇數
以計算29000個文字的出現篇數
以計算30000個文字的出現篇數
以計算31000個文字的出現篇數
以計算32000個文字的出現篇數
以計算33000個文字的出現篇數
以計算34000個文字的出現篇數
以計算35000個文字的出現篇數
以計算36000個文字的出現篇數
以計算37000個文字的出現篇數
以計算38000個文字的出現篇數
以計算39000個文字的出現篇數
以計算40000個文字的出現

In [42]:
contentcutlist[1]

'原文 台南 東區 深藍咖啡館 號稱 蛋糕 界 千層 蛋糕 大人 口味 黑森林 蛋糕 早茶 在當 貴婦 普普 時尚 風格 超像 藝文 咖啡館 女子 休假 計劃 新加坡 吃 好吃 喜歡 香嫩 蛋皮裡 裏 慕斯 香甜 滋味 聽說 台南 厲害 號稱 蛋糕 界 千層 蛋糕 選 早茶 時段 前來 進門 冷氣 還蠻強 冰櫃 透明 窗上 結上 一層 水氣 店內 呈現 普普 風格 給人 現代 時尚 感 一致性 水藍及 白色 系列 設計 主軸 設計 感 空間 寬敞 自覺 有點 來到 藝文 咖啡館 外帶 盒 普普 圓點 風格 年 獲得 紅點 大賞 送禮 自已 提 回家 吃 氣派 早茶 時段 蛋糕 栗子 千層 蛋糕 再 加碼 挑 最愛 吃 黑森林 蛋糕 早茶 蛋糕 提供 一份 任選 蛋糕 一壺 茶 黑森林 蛋糕 荔汁 白茶 選用 美國 有機 茶 含 咖啡因 茶 包泡 茶味 不香 荔汁味 水味 多於 茶味 有點 失望 黑森林 蛋糕 表層 酒漬 過後 黑 櫻桃 配上 微苦 巧克力 片 底層 慕絲 層層 堆疊 濕潤 蛋糕 體 奶油 空氣 感 柔順 湯匙 挖至 底層 絕妙 大人 成熟 滋味 非常好吃 栗子 千層 蛋糕 網友 號稱 蛋糕 界 千層 蛋糕 高度 薄度 忍不住 數了 一下 聽說 店家 已經 厲害 製成 多層 唷 細緻 柔順 法國 精選 栗子 熬 煮成 醬 舖 表層 另將 奶油 香甜 栗子 一層 一層 費工 舖 一片 蛋皮 放在 冰櫃 太久 蛋皮略 硬 有點 可惜 結論 挑 早午 茶 時段 前來 吃 千層 蛋糕 新鮮 現做 服務態度 挺 好 認真 介紹 深藍咖啡館 網址 地址 台南市 府連 東路 號 電話 營業時間 週一 週日 咖啡 茶 飲品 蛋糕 免 服務費 無 限時 插座 低銷 一份 餐點 檢視 較大 地圖 早上 八點 營業 深藍咖啡館 網友 說 全台 最 好吃 千層 蛋糕 知名 美食 節目 報導 每日 定時 供應 種 口味 千層 若想 吃 愛的 口味 碰碰運氣 文字 男子 攝影 男子 深藍咖啡館 男子 日常生活 男子 日常生活 台南 時間 決定 中午 前 先 過來 原本 以為 一大 早來 不用擔心 吃 不到 想 吃 口味 原來 深藍咖啡館 每個 時段 只 提供 部分 口味 固定 時間 提供 一種 口味 想要 吃 想 吃 口味 真的 看運氣 焦糖 原味 最 想 品嚐 失望 焦糖 千層

# 將上面的斷詞文章combine在一起 做word2vec model

In [9]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(contentcutlist)
with open('D:\Data\word2VecData\TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("D:/Data/word2VecData/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("D:/Data/word2VecData/Tainan.model.bin")


2017-09-06 13:54:14,575 : INFO : collecting all words and their counts
2017-09-06 13:54:14,575 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-09-06 13:54:16,096 : INFO : collected 170075 word types from a corpus of 6445987 raw words and 645 sentences
2017-09-06 13:54:16,112 : INFO : Loading a fresh vocabulary
2017-09-06 13:54:16,380 : INFO : min_count=4 retains 53503 unique words (31% of original 170075, drops 116572)
2017-09-06 13:54:16,380 : INFO : min_count=4 leaves 6285274 word corpus (97% of original 6445987, drops 160713)
2017-09-06 13:54:16,517 : INFO : deleting the raw counts dictionary of 170075 items
2017-09-06 13:54:16,517 : INFO : sample=0.001 downsamples 17 most-common words
2017-09-06 13:54:16,517 : INFO : downsampling leaves estimated 6118823 word corpus (97.4% of prior 6285274)
2017-09-06 13:54:16,517 : INFO : estimated required memory for 53503 words and 300 dimensions: 155158700 bytes
2017-09-06 13:54:16,684 : INFO : resetting layer we

2017-09-06 13:55:29,201 : INFO : PROGRESS: at 61.89% examples, 263345 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:30,250 : INFO : PROGRESS: at 62.79% examples, 263352 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:31,254 : INFO : PROGRESS: at 63.69% examples, 263527 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:32,290 : INFO : PROGRESS: at 64.56% examples, 263428 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:33,310 : INFO : PROGRESS: at 65.46% examples, 263510 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:34,331 : INFO : PROGRESS: at 66.29% examples, 263338 words/s, in_qsize 7, out_qsize 0
2017-09-06 13:55:35,349 : INFO : PROGRESS: at 67.16% examples, 263344 words/s, in_qsize 8, out_qsize 0
2017-09-06 13:55:36,352 : INFO : PROGRESS: at 68.06% examples, 263466 words/s, in_qsize 8, out_qsize 0
2017-09-06 13:55:37,467 : INFO : PROGRESS: at 68.99% examples, 263371 words/s, in_qsize 8, out_qsize 0
2017-09-06 13:55:38,470 : INFO : PROGRESS: at 69.89% examples, 263513 wor

In [40]:
len(contentcutlist)

5768

# 試玩word2vec的model  需解除#

In [41]:
from gensim.models import word2vec
from gensim import models
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = models.Word2Vec.load('D:/Data/word2VecData/Tainan.model.bin')

print("提供 3 種測試模式\n")
print("輸入一個詞，則去尋找前一百個該詞的相似詞")
print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
print("輸入三個詞，進行類比推理")

while True:
    try:
        query = input()
        q_list = query.split()

        if len(q_list) == 1:
            print("相似詞前 30 排序")
            res = model.most_similar(q_list[0],topn = 30)
            for item in res:
                print(item[0]+","+str(item[1]))

        elif len(q_list) == 2:
            print("計算 Cosine 相似度")
            res = model.similarity(q_list[0],q_list[1])
            print(res)
        else:
            print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
            res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
            for item in res:
                print(item[0]+","+str(item[1]))
        print("----------------------------")
    except Exception as e:
        print(repr(e))


2017-09-06 14:53:11,815 : INFO : loading Word2Vec object from D:/Data/word2VecData/Tainan.model.bin
2017-09-06 14:53:11,903 : INFO : loading wv recursively from D:/Data/word2VecData/Tainan.model.bin.wv.* with mmap=None
2017-09-06 14:53:11,903 : INFO : loading syn0 from D:/Data/word2VecData/Tainan.model.bin.wv.syn0.npy with mmap=None
2017-09-06 14:53:11,934 : INFO : setting ignored attribute syn0norm to None
2017-09-06 14:53:11,934 : INFO : loading syn1neg from D:/Data/word2VecData/Tainan.model.bin.syn1neg.npy with mmap=None
2017-09-06 14:53:11,981 : INFO : setting ignored attribute cum_table to None
2017-09-06 14:53:11,982 : INFO : loaded D:/Data/word2VecData/Tainan.model.bin


提供 3 種測試模式

輸入一個詞，則去尋找前一百個該詞的相似詞
輸入兩個詞，則去計算兩個詞的餘弦相似度
輸入三個詞，進行類比推理
好吃


2017-09-06 14:53:19,326 : INFO : precomputing L2-norms of word weight vectors


相似詞前 30 排序
很好吃,0.640457034111023
非常好吃,0.5930620431900024
難吃,0.5671696066856384
超好吃,0.5651984214782715
超刷,0.5438476800918579
不好吃,0.5428752303123474
速配,0.5349238514900208
整個來說,0.533810019493103
比較滿意,0.5325368642807007
合味,0.5314146876335144
火爆,0.5313247442245483
份炸,0.5310989618301392
第七道,0.5277280807495117
合得來,0.5269577503204346
油裡面,0.5244425535202026
實在話,0.5243936777114868
膩吃,0.5242106318473816
超超超,0.5232260227203369
飯超,0.5231798887252808
貴點,0.5223608016967773
料算,0.5221067667007446
過炸,0.5219387412071228
一吃會,0.5210387706756592
禁得起,0.5208934545516968
炸機,0.5201650857925415
包君滿意,0.5193314552307129
第三盤,0.518811821937561
跟意,0.518717885017395
變乾,0.5181980729103088
還挺合,0.5179552435874939
----------------------------
雞腿 豆花
計算 Cosine 相似度
0.0552919024157
----------------------------
1


KeyboardInterrupt: 

# 試Rocchio、bayes、kmeans、SVM分群 (仿白話大數據寫法)

In [11]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(contentcutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')

#ptt測試集
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
# from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)

#試用bayes分群
# from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)

#試用kmeans分群
# from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Kmeans")
pprint.pprint(predicted)

#試用svm分群
# from sklearn import svm
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("svm")
pprint.pprint(predicted)

文字轉向量矩陣
tfidf
Roccio
array(['台菜餐廳', '美式料理', '義式料理', '西點烘焙、麵包', '其他鍋類', '其他小吃', '臭豆腐', '',
       '義式料理', '美式料理', '其他小吃', '泰式料理', '台菜餐廳'],
      dtype='<U13')
bayes
array(['複合式', '複合式', '複合式', '複合式', '複合式', '其他小吃', '其他小吃', '其他小吃', '複合式',
       '複合式', '其他小吃', '其他小吃', '複合式'],
      dtype='<U13')
Kmeans
array(['其他小吃', '複合式', '複合式', '酒吧 Lounge Bar', '複合式', '美式料理', '其他小吃', '中式',
       '複合式', '其他小吃', '飲料專賣店', '酒吧 Lounge Bar', '飲料專賣店'],
      dtype='<U13')
svm
array(['其他小吃', '複合式', '義式料理', '西點烘焙、麵包', '其他小吃', '其他小吃', '臭豆腐', '其他小吃',
       '複合式', '其他小吃', '其他小吃', '泰式料理', '其他小吃'],
      dtype='<U13')


# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > kmeans > bayes

In [12]:
#參考PTT斷詞文章
docs_new

['食記 壹等賞 桃園 龍潭 看板 批 踢踢 實業 坊 批 踢踢 實業 坊 看板 聯絡 資訊 返回 看板 分享 作者 看板 標題 食記 壹等賞 桃園 龍潭 時間 本文 轉錄 看板 作者 看板 標題 食記 壹等賞 桃園 龍潭 時間 圖文 版 今天 提早到 新竹 火車站 附近 髮 瑟 剪 頭髮 整個 人 變得 清爽 免不了 念 幾句 想 以後 頭髮 剪 好 今天 比較 晚 下班 六點 多才 離開 公司 交通 正 處於 尖峰 時刻 光復路 繞 一圈 還了 漫畫 之後 決定 晚上 吃 桃園 龍潭 壹等賞 新竹 文化 街上 紅葉 告訴 目前 肚子 不餓 想 說 現在 開 壹等賞 時間 應該 夠長 肚子 應該 餓 決定 上次 曾經 吃過 一次 壹等賞 享用 晚餐 壹等賞 位於 龍潭 渴望 園區 附近 上次 來過 一次 不知 道路 走 知道 高速公路 累得 睡著 開 十分鐘 下 關西 交 流道 辦法 睡 一下 一下 交流 道 吵醒 請 幫 帶路 今 天 再 走過 一次 不太 認得 路 晚上 天色 黑 另一方 面是 山路 白天 去過 難記 住 路 走 下面 名片 背後 地圖 有興 趣 參考 一下 地址 桃園縣 龍潭 鄉 高原 村 中原 路 二段 巷 弄 號 電話 營業時間 每週 公休 今天 一到 停車場 滿滿的 車子 已經 七點 多了還 一堆 人 一看 發現 應該 公司 聚餐 很多人 穿著 同樣 制服 外面 抽 飯後 煙 空間 很大 晚到 位置 坐 壹等賞 一個 茶園 旁邊 一片 很大 茶 田 裡面 餐廳 賣 茶葉 東西 就連 洗手間 洗手 乳 茶葉 香味 服務 人員 一間 廂房 裡面 已經 坐著 滿滿的 客人 一到 看到 客人 本來 已經 心理準備 廚房 上菜 比較慢 一點 一點 完菜 不到 三分鐘 菜 一道 一道 送上來 真 佩服 廚房 裡面 大廚 燒菜 功力 上菜 食物 好吃 熱茶 免費 餐桌上 一壺 大壺 熱茶 免費 提供 飲用 醃菜 頭 免費 喜歡 吃 只 吃 兩塊 剩下 吃 光光 用來 搭配 稀飯 一起 吃 最好 稀飯 點 好吃 茶油 拌 飯 珍香蜜 茶 鵝 小 茶鵝 很好吃 有點 貴 外面 賣 鵝肉 不太 應該 有用 茶葉 燻 很香 養生茶 油拌 飯 點了 兩碗 四碗 普通 碗 份量 要點 茶油 拌 飯 吃 哪有 吃白飯 道理 桂花 蜜 排骨 小 糖醋排骨 吃 覺得 

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算10個)

In [13]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
pinCountMaxper5list = []
for diencomment in diencommentlist:
    dienWordcount={}
    for comment in  diencomment:
        for pinword in pinlist:
            if len(re.findall(pinword,comment))>0:
                if len(re.findall(pinword,comment))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,comment))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)



In [14]:
#店家評論總數Ncommentlist

#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Ncommentlist):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [52]:
 dienscorelist[60]

34.219999999999985

In [15]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [54]:
pinsynonymsdict

{'CP值很低': '否划算',
 'Q彈': '食物美味',
 'cp值不高': '否划算',
 'cp值很高': '划算',
 'hen好吃': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一般般': '否食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美味',
 '不親民': '否划算',
 '不難吃': '食物美味',
 '並不貴': '划算',
 '中低': '划算',
 '也不錯': '食物美味',
 '乾了點': '否食物美味',
 '乾淨': '環境好',
 '亂': '否環境好',
 '亂掉': '否服務好',
 '令人愉快': '環境好',
 '令人滿意': '服務好',
 '令人驚豔': '食物美味',
 '份量大': '份量大',
 '但貴': '否划算',
 '低廉': '划算',
 '佛心': '划算',
 '你絕對不會想': '否食物美味',
 '俗擱': '份量大',
 '值不高': '否划算',
 '值夠': '划算',
 '值得稱讚': '服務

In [16]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Ncommentlist):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

In [61]:
featurescores[1]

{'份量大': 0.07,
 '偏甜': 0.58,
 '其他正面情緒': 0.83,
 '划算': -0.63,
 '小份量': 0.1,
 '服務好': -0.09,
 '環境好': 0.38,
 '食物很香': 0.61,
 '食物精緻漂亮': 0.49,
 '食物美味': 0.56}

# 將上述屬性結合成一個大分析表

In [17]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore in zip(dienlist,stylelist,Ncommentlist,contentcutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# contentcutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper3list
# dienscorelist
# featurescores

In [60]:
BigAnalyzeTable[0]

{'Ncomment': 20,
 'TF_IDF': Counter({'一下': 0.0005634393331149615,
          '一個': 0.0008139148100223864,
          '一口': 0.0013077527366750165,
          '一定': 0.0005084299198928792,
          '一家': 0.0011237862149287283,
          '一張': 0.0014579745415025532,
          '一杯': 0.0010869173871837863,
          '一次': 0.0008449702463452307,
          '一碗': 0.004429666659221711,
          '一種': 0.0009967091312932672,
          '一起': 0.0011374331969887345,
          '一間': 0.0006565365312943577,
          '一點': 0.0012219725111579588,
          '三種': 0.0011375225600803959,
          '上次': 0.0015558003981493821,
          '上面': 0.0006968974079722043,
          '下': 0.0001949567919716747,
          '不用': 0.0009490672845530912,
          '不知': 0.0010662962830992676,
          '中午': 0.0011840833079356117,
          '久': 0.0005288374071046689,
          '之後': 0.0007200937979180292,
          '二樓': 0.0017070511683660698,
          '人': 0.0012173607844583785,
          '人潮': 0.0028807927880091623,
  

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [18]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	31.0	-19.0
食物美味+5	16.7	0
食物美味+10	18.1	0
食物美味+15	14.9	0
划算+0	8.0	-10.0
划算+5	3.2	-5.1
划算+10	3.3	-1.8
划算+15	2.1	-1.7
環境好+0	21.0	-3.0
環境好+5	4.4	-0.5
環境好+10	5.0	-0.3
環境好+15	3.8	-0.1
服務好+0	3.0	-2.0
服務好+5	1.9	-0.3
服務好+10	0.9	-0.3
服務好+15	0.9	-0.2
食物精緻漂亮+0	13.0	0
食物精緻漂亮+5	6.7	0
食物精緻漂亮+10	5.9	0
食物精緻漂亮+15	5.1	0
食物很香+0	6.0	0
食物很香+5	3.0	0
食物很香+10	4.4	0
食物很香+15	3.1	0
其他正面情緒+0	7.0	-2.0
其他正面情緒+5	2.1	-0.3
其他正面情緒+10	4.6	-0.2
其他正面情緒+15	1.1	-0.1
份量大+0	7.0	0
份量大+5	2.3	0
份量大+10	2.6	0
份量大+15	2.9	0
小份量+0	5.0	0
小份量+5	2.9	0
小份量+10	2.9	0
小份量+15	2.1	0
重口味+0	5.0	0
重口味+5	2.2	0
重口味+10	1.0	0
重口味+15	1.0	0
口味清淡+0	3.0	0
口味清淡+5	0.6	0
口味清淡+10	0.4	0
口味清淡+15	1.2	0
偏辣+0	5.0	0
偏辣+5	1.3	0
偏辣+10	1.6	0
偏辣+15	0.8	0
偏甜+0	3.3	0
偏甜+5	1.2	0
偏甜+10	0.8	0
偏甜+15	1.2	0
偏酸+0	1.0	0
偏酸+5	0.3	0
偏酸+10	0.3	0
偏酸+15	0.3	0


In [19]:
standard

{'份量大0': (3.5, 3.5),
 '份量大10': (1.3, 1.3),
 '份量大15': (1.45, 1.45),
 '份量大5': (1.15, 1.15),
 '偏甜0': (1.65, 1.65),
 '偏甜10': (0.4, 0.4),
 '偏甜15': (0.6, 0.6),
 '偏甜5': (0.6, 0.6),
 '偏辣0': (2.5, 2.5),
 '偏辣10': (0.8, 0.8),
 '偏辣15': (0.4, 0.4),
 '偏辣5': (0.65, 0.65),
 '偏酸0': (0.5, 0.5),
 '偏酸10': (0.15, 0.15),
 '偏酸15': (0.15, 0.15),
 '偏酸5': (0.15, 0.15),
 '其他正面情緒0': (2.5, 4.5),
 '其他正面情緒10': (2.1999999999999997, 2.4),
 '其他正面情緒15': (0.5, 0.6000000000000001),
 '其他正面情緒5': (0.9, 1.2000000000000002),
 '划算0': (-1.0, 9.0),
 '划算10': (0.7499999999999999, 2.55),
 '划算15': (0.20000000000000007, 1.9),
 '划算5': (-0.9499999999999997, 4.15),
 '口味清淡0': (1.5, 1.5),
 '口味清淡10': (0.2, 0.2),
 '口味清淡15': (0.6, 0.6),
 '口味清淡5': (0.3, 0.3),
 '小份量0': (2.5, 2.5),
 '小份量10': (1.45, 1.45),
 '小份量15': (1.05, 1.05),
 '小份量5': (1.45, 1.45),
 '服務好0': (0.5, 2.5),
 '服務好10': (0.30000000000000004, 0.6),
 '服務好15': (0.35, 0.55),
 '服務好5': (0.7999999999999999, 1.1),
 '環境好0': (9.0, 12.0),
 '環境好10': (2.35, 2.65),
 '環境好15': (1.8499999999999999, 1

In [20]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [21]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	1.2	-0.8
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
划算+0	0.9	-1.1
划算+5	0.8	-1.2
划算+10	1.3	-0.7
划算+15	1.1	-0.9
環境好+0	1.8	-0.2
環境好+5	1.8	-0.2
環境好+10	1.9	-0.1
環境好+15	2.0	-0.1
服務好+0	1.2	-0.8
服務好+5	1.7	-0.3
服務好+10	1.5	-0.5
服務好+15	1.6	-0.3
食物精緻漂亮+0	2.0	0
食物精緻漂亮+5	2.0	0
食物精緻漂亮+10	2.0	0
食物精緻漂亮+15	2.0	0
食物很香+0	2.0	0
食物很香+5	2.0	0
食物很香+10	2.0	0
食物很香+15	2.0	0
其他正面情緒+0	1.6	-0.4
其他正面情緒+5	1.8	-0.2
其他正面情緒+10	1.9	-0.1
其他正面情緒+15	1.8	-0.2
份量大+0	2.0	0
份量大+5	2.0	0
份量大+10	2.0	0
份量大+15	2.0	0
小份量+0	2.0	0
小份量+5	2.0	0
小份量+10	2.0	0
小份量+15	2.0	0
重口味+0	2.0	0
重口味+5	1.9	0
重口味+10	2.0	0
重口味+15	2.0	0
口味清淡+0	2.0	0
口味清淡+5	1.9	0
口味清淡+10	2.1	0
口味清淡+15	2.0	0
偏辣+0	2.0	0
偏辣+5	2.0	0
偏辣+10	2.0	0
偏辣+15	2.0	0
偏甜+0	2.0	0
偏甜+5	1.9	0
偏甜+10	2.0	0
偏甜+15	2.0	0
偏酸+0	2.0	0
偏酸+5	2.2	0
偏酸+10	2.2	0
偏酸+15	2.3	0


# 貼餐廳評價標籤!

In [22]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        
        if dien["featurescores"][feature]>1:
            for pinpare in pipin:
                if feature==pinpare[0]:
                    newtag.append(pinpare[2])
        elif dien["featurescores"][feature]>0.5:
            newtag.append(feature)
        elif dien["featurescores"][feature]<-0.5:
            for pinpare in pipin:
                if feature==pinpare[0]:
                    newtag.append(pinpare[3])
        elif dien["featurescores"][feature]<0.0:
            for pinpare in pipin:
                if feature==pinpare[0]:
                    newtag.append(pinpare[1])
                    
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

同記安平豆花(安平總店)
['豆花', '安平', '珍珠', '同記安平豆花', '紅豆', '剉冰、豆花']
深藍咖啡館
['服務較差', '偏甜', '栗子', '食物美味', '非常貴', '千層', '食物很香', '其他正面情緒', '蛋糕', '一層', '深藍', '西點烘焙、麵包']
阿堂鹹粥
['價格較高昂', '油條', '女孩', '食物美味', '粥', '小份量', '虱目', '魚肚', '其他小吃']
富盛號碗粿
['粿', '魚羹', '富盛號', '碗', '永樂', '其他小吃']
依蕾特布丁奶酪(安平門市)
['奶酪', '價格較高昂', '抹茶', '典藏', '依蕾特', '布丁', '西點烘焙、麵包']
食下有約 想法廚房
['花雕', '制式', '環境超棒', '食下', '想法', '其他正面情緒', '非常好吃', '蹄膀', '其他異國料理']
飛饗創意西式料理
['價格較高昂', '燉', '網路評論正面情緒非常多', '土壤', '偏酸', '環境超棒', '典藏', '沙拉', '服務好', '份量大', '非常好吃', '饗', '義式料理']
莉莉水果店
['前路', '價格較高昂', '水果', '小份量', '莉莉', '切盤', '西瓜', '剉冰、豆花']
連得堂餅舖
['價格較高昂', '兩包', '煎餅', '崇安', '餅家', '連得堂', '休閒零食']
矮仔成蝦仁飯
['價格較高昂', '飯', '評論有些負面情緒詞', '蛋湯', '鴨蛋', '矮', '蝦仁', '小份量', '其他小吃']
泰成水果店
['正興', '價格較高昂', '瓜瓜', '食物美味', '哈蜜瓜', '水果', '其他正面情緒', '食物精緻漂亮', '哈密瓜', '果汁']
騷烤家(前鋒總店)
['甜不辣', '價格較高昂', '油條', '烤', '食物很香', '其他正面情緒', '燒烤', '非常好吃', '騷烤', '重口味', '食物精緻漂亮', '炭烤串燒']
阿松割包
['價格較高昂', '評論有些負面情緒詞', '豬舌', '割包', '瘦肉', '豬舌包', '酸菜', '麵食點心']
秉醇烘焙坊
['服務較差', '偏甜', '網路評論正面情緒非常多', '餅乾', '食物美味'

木門朝午洋食 MOMENT
['木門', '價格較高昂', '佐藤', '鬆餅', '洋食', '朝午', '其它主題餐廳']
來呷麵
['魯味', '老闆娘', '到後甲國', '補習', '切個', '麵攤、麵店']
TWIN coffee 咖啡 云
['云', '價格較高昂', '偏甜', '環境好', '鬆餅', '偏酸', '食物美味', '食物很香', '崇明', '小份量', '庫克', '咖啡', '食物精緻漂亮', '複合式']
Dazzling Cafe(台南新光三越中山店)
['蜜糖', '價格較高昂', '偏甜', '服務較差', '草莓', '吐司', '食物美味', '新光', '冰淇淋', '複合式']
新都巷日本料理
['牡丹', '生魚片', '竹定', '食物美味', '壽司', '份量大', '大洋', '划算', '食物精緻漂亮', '綜合日式料理']
方蘭川焦皮布丁
['服務較差', '偏甜', '窯', '安平', '食物很香', '焦糖', '布丁', '焦皮', '西點烘焙、麵包']
Picturesque 塗鴉空間書店(長榮2店)
['價格較高昂', '塗鴉', '書店', '帕里尼', '野鴨', '空間', '食物美味', '環境超棒', '食物很香', '其他正面情緒', '食物精緻漂亮', '其它主題餐廳']
Daylight光合箱子
['價格較高昂', '箱子', '吐司', '食物美味', '豬小排', '光合', '早午餐', '複合式']
德斯啤鮮釀啤酒餐廳
['德國', '價格較高昂', '環境好', '德斯', '鮮釀', '其他正面情緒', '非常好吃', '偏辣', '豬腳', '啤酒', '德式料理']
秋收碳烤
['實攤', '價格較高昂', '評論有些負面情緒詞', '關東煮', '秋收', '荖', '碳', '其他小吃']
Yellow Bee手創漢堡
['服務較差', '漢堡', '雞腿', '食物美味', '手創', '重口味', '食物精緻漂亮', '堡', '手套', '美式料理']
1004 Brunch 早午餐
['勾咕', '環境好', '早午餐', '帕里尼', '韓國', '食物很香', '其他正面情緒', '非常好吃', '天使', '食物精緻漂亮', '複合式'

鳳凰來手作炸雞屋
['長榮', '三角', '魷魚', '食物美味', '食物很香', '燒烤', '雞排', '鹹酥雞、炸雞排']
小草堂
['價格較高昂', '偏甜', '環境好', '豆乳', '玫瑰花', '蛋糕', '新美街', '凡爾賽', '複合式']
游爸爸蕃薯椪 白糖粿
['粿', '椪', '白糖', '食物很香', '小份量', '蕃薯', '游', '休閒零食']
Mr.Wheat 小麥先生創意料理(文平義式館)
['文平義式', '偏甜', '環境好', '價格較高昂', '先生', '小麥', '其他正面情緒', '小春', '非常好吃', '金華', '義式料理']
CHEERYWOOD 櫻桃木
['太妃', '崇善路', '價格較高昂', '環境好', '食物美味', '洋房', '公寓', '櫻桃木', '食物精緻漂亮', '複合式']
不老蒸氣養生料理
['環境好', '蒸煮', '沙包', '粥', '蟳', '蒸氣', '食物很香', '其他正面情緒', '份量大', '非常好吃', '海鮮餐廳']
奇美咖啡館(成大店)
['咖啡館', '迪克', '義大利', '布丁', '奇美', '咖啡專賣']
TAIKOO Cafe 太古咖啡
['班尼', '迪克', '份量大', '東門路', '太古', '咖啡店', '咖啡專賣']
讚岐廚房
['價格較高昂', '偏酸', '食物美味', '小幽', '祇', '烏龍', '蒲小英', '豬排', '其他日式料理']
澄花日式壽喜燒(台南店)
['壽喜鍋', '菜盤', '鍋底', '壽喜燒', '澄花', '壽喜燒']
湯咖哩
['咖哩', '野菜', '札幌', '普通型', '觀亭', '其他日式料理']
阿魯香腸熟肉
['價格較高昂', '切料', '服務較差', '評論有些負面情緒詞', '環境比較不好', '小份量', '熟肉', '香腸', '阿魯', '粉腸', '其他小吃']
安平貴記美食文化館
['粿', '肉燥', '價格較高昂', '肉圓', '趖', '鼎', '其他小吃']
歐貝拉(台南支店)
['喜餅', '歐', '草莓', '貝拉', '蛋糕', '西點烘焙、麵包']
永記虱目魚丸
['肉燥', '魚丸', '價格較高昂', '環境好', '永記

['剉冰', '江水號', '四果', '泰山', '圓仔', '飲料專賣店']
葳林爵閣 Winnie Drinker
['飲品', '葳林', '服務較差', '價格較高昂', '奶蓋', '爵閣', '食物很香', '金莎', '複合式']
畢夫餅BEAVERPASTRY
['安平', '畢夫', '加拿大', '小份量', '霜淇淋', '畢夫餅', '西點烘焙、麵包']
Cafe' LOFT13 咖啡樓閣
['流暢', '樓閣', '價格較高昂', '嗅不出', '虹吸式', '二樓', '複合式']
岩本大阪燒
['價格較高昂', '忠良', '日本', '鐵板', '大阪', '岩本', '文字燒、大阪燒']
田媽媽ㄋㄟㄋㄟ寶(柳營鄉農會)
['饅頭', '田', '玉梅', '山藥', '包子', '其他小吃']
西品經濟便當
['便當', '價格較高昂', '控肉', '火燒', '蝦仁', '西港', '其他小吃']
億居酒屋
['價格較高昂', '海膽', '偏酸', '鹽燒', '甘藍', '優格', '觀亭', '居酒屋']
小碗和食
['評論有些負面情緒詞', '大安', '梅子', '日式', '食物美味', '東區', '味自慢', '綜合日式料理']
一串心燒烤屋
['一串', '神農', '環境比較不好', '海安', '人行道', '重口味', '炒雞蛋', '炭烤串燒']
樂烘烘 一起玩烘焙
['樂烘', '甜點', '平板', '烤箱', '材料', '西點烘焙、麵包']
炒翻天(育樂店)
['三鮮', '乙町翁', '炒飯', '翻天', '紅樓小館', '台菜餐廳']
麥當勞(台南麻豆店)
['麥當當', '噗', '麥當勞', '喵', '薯餅', '漢堡、炸雞']
老城門洞 - 重慶麻辣鍋(台南安平店)
['芍粉', '麻辣鍋', '重慶', '城門洞', '小夥伴', '麻辣火鍋']
丼丸
['丼', '販賣機', '燒起', '東區', '食物很香', '富士山', '其他日式料理']
山林香 泰國辛香小吃
['酸辣', '咖哩', '朕', '泰國', '山林', '泰式料理']
食大客平價炙燒牛排
['南紡', '盎司', '購物中心', '大丈夫', '牛排', '划算', '其他異國料理']
庫瑪の家
[

梅鳳飲食店
['老爸', '辦桌', '中炎山', '後壁', '炎山', '台菜餐廳']
町之戶 デザート屋
['粉嫩', '紅鶴', '之戶', '泳圈', '町', '食物精緻漂亮', '西點烘焙、麵包']
安堤生烘焙
['玻璃瓶', '冰心', '吐司', '安堤', '布丁', '西點烘焙、麵包']
Liberal café  自由派
['自由派', '信義', '巧克力', '草莓', '塔', '複合式']
熊本式拉麵專賣店
['式豚', '熊本', '鰹魚', '白湯', '拉麵', '日式麵食專賣']
WO TANG XING 莪堂行
['莪', '老宅', '昆沙宮', '帕尼尼', '堂行', '複合式']
洪家店牛肉麵專賣
['細菌', '牛肉麵', '酸菜', '洪家', '破豆', '麵食點心']
銀波茶座
['奶酪', '價格較高昂', '銀波', '美特', '銀波貝', '布丁', '西點烘焙、麵包']
鹿府 LU FU(二館)
['參館', '鹿府', '劍獅', '海山', '貳', '複合式']
末廣町侍女隊女傭主題餐廳
['價格較高昂', '女僕', '漫畫', '主人', '阿宅', '女傭', '其它主題餐廳']
馬來峰巴生肉骨茶
['雞飯', '海南', '馬來西亞', '僑生', '肉骨', '其他異國料理']
冠軍雞碳烤屋
['價格較高昂', '冠軍', '黑輪', '環境比較不好', '食物很香', '肉串', '胗', '老板娘', '炭烤串燒']
花川日式料理
['生魚片', '魚拓', '花川日本料理', '中食', '小高', '綜合日式料理']
合伙燒日式關東煮燒烤
['明太子', '合伙', '食物美味', '關東煮', '尊王路', '食物很香', '七里香', '居酒屋']
台南大飯店-翡翠廳
['價格較高昂', '麗絲', '大飯店', '牛排', '麗絲家', '閤', '西式']
翰林茶館(赤崁店)
['價格較高昂', '茶館', '粉圓', '翰林', '十七年', '珍珠奶茶', '果汁']
鹽鄉民宿
['弄得好', '絲瓜', '鹽田', '香腸', '虱目', '海鮮餐廳']
棠棠肉品
['夾雜', '土司', '東窗事發', '肉片', '烤烤', '其他類型早餐']
金福氣南洋食

['價格較高昂', '和緯店', '焗', '香蒜', '弟弟', '伊甸風味館', '義式料理']
舊永瑞珍喜餅
['特訂', '烏豆', '喜餅', '西餅', '舊永瑞珍', '西點烘焙、麵包']
泰好水餃店
['煎餃', '爸比', '水餃', '登', '銘哥', '其他小吃']
黃火木冰店
['八寶', '江水號', '台南美食', '黃火木', '食物美味', '老先生', '剉冰、豆花']
盛來炒飯店
['華興', '回鍋肉', '炒飯', '口味清淡', '仔略', '蒜苗', '熱炒、快炒']
韓氏古早味油飯
['價格較高昂', '韓氏', '一斤', '油飯', '節目', '一集', '麵食點心']
天津牛肉麵
['太軟', '翡翠', '條糊掉', '牛肉麵', '豬腳', '其他小吃']
梅子蔬食
['五穀', '焗', '堅果', '素食', '疏食', '西式']
Cheese & Risotto
['雙料', '摩勒特', '胖拔', '自拉', '乳酪', '其他小吃']
咔滋咔滋日式豬排定食
['蒸蛋', '培煎', '口卡滋口', '豬排', '搗碎', '日式豬排專賣']
梅嶺食品中心
['價格較高昂', '話梅', '青梅', '試吃', '一桶', '梅嶺', '休閒零食']
許家點心店
['小雞', '點的餐', '炒飯', '服務好', '講它', '看鍋燒', '划算', '台菜餐廳']
查理Tea Shop
['查理', '融合', '可拿滋', '環境比較不好', '口味清淡', '多拿滋', '東安', '食物精緻漂亮', '複合式']
郭家米糕
['那烏', '喝價', '沒什人', '湯杯', '米糕', '筒仔米糕']
嘉昌雞肉飯(新市店)
['嘉昌雞肉飯', '泡飯', '菜頭', '鮮加', '連我住', '其他小吃']
富香齋餅舖
['綠豆糕', '香齋', '塩', '澎餅', '香餅', '西點烘焙、麵包']
沐春溫泉養生會館
['教堂', '荷風', '泉質', '泡湯', '環境比較不好', '溫泉', '溫泉餐廳']
正宗鮮蝦餅
['手禮給', '螃蟹', '趕回來', '蝦餅', '塞辦', '西點烘焙、麵包']
牛狀元牛肉湯
['杓子', '雪倫', '狀元', '牛肉', '環境比

['', '無評', '泰式料理']
薛師傅當歸鴨餐廳
['', '無評', '食補藥膳餐廳']
香積坊素食
['', '無評', '中式']
貝葉蓮心素食
['', '無評', '中式']
家樂蛋糕
['', '無評', '西點烘焙、麵包']
土耳其之屋
['', '無評', '其他異國料理']
北安豬牛排
['', '無評', '美式料理']
綠堡花草庭園
['', '無評', '咖啡專賣']
府城海產
['', '無評', '海鮮餐廳']
農夫創意餐飲
['', '無評', '其他鍋類']
夜園冰室
['', '無評', '冰淇淋']
東家牛肉麵
['', '無評', '麵食點心']
妙手刀削麵
['', '無評', '麵食點心']
新溪邊土雞城
['', '無評', '山產野菜餐廳']
上新薑母鴨
['', '無評', '薑母鴨']
昆旺羊肉店
['', '無評', '食補藥膳餐廳']
萬得福平價牛排
['', '無評', '美式料理']
菘苗素食
['', '無評', '中式']
緣園素食(民生店)
['', '無評', '中式']
三元齋素食
['', '無評', '中式']
夏慕兒果子烘培
['', '無評', '西點烘焙、麵包']
阿官火鍋專家(永康中華加盟店)
['', '無評', '其他鍋類']
府城將軍雞腳凍(佳里忠孝店)
['', '無評', '休閒零食']
陋巷咖啡
['', '無評', '咖啡專賣']
首相大飯店
['', '無評', '其它中式料理']
娟記大餅
['', '無評', '其他小吃']
里麻里肉圓專賣店
['', '無評', '其他小吃']
異色引誘戀豆坊
['', '無評', '西點烘焙、麵包']
奉茶來恁兜(台南南門)
['', '無評', '飲料專賣店']
Nice尼斯歐義式餐坊
['', '無評', '義式料理']
久昌飯店
['', '無評', '熱炒、快炒']
劉太太手工水餃
['', '無評', '麵食點心']
阿文師蒜香排骨飯
['', '無評', '熱炒、快炒']
饕公麻辣燙(永康店)
['', '無評', '其他小吃']
春妹越南法國麵包(海佃)
['', '無評', '越南料理']
糧倉碳烤
['', '無評', '炭烤串燒']
町仔腳88碳烤善化總店
['', '無評', '炭烤串燒']
皇家牛排風

['', '無評', '飲料專賣店']
麥當勞(台南中華二)
['', '無評', '漢堡、炸雞']
酌影居酒屋
['', '無評', '居酒屋']
湠咖啡
['', '無評', '咖啡專賣']
小蒙牛頂級麻辣養生鍋(仁德店)
['', '無評', '麻辣火鍋']
家鄉碳烤香雞排
['', '無評', '鹹酥雞、炸雞排']
狐狸尾巴(金華店)
['', '無評', '飲料專賣店']
悠然亭
['', '無評', '咖啡專賣']
仙茶舖
['', '無評', '飲料專賣店']
柳營黑白切
['', '無評', '麵攤、麵店']
多樂餐飲
['', '無評', '其他類型早餐']
味鄉簡速快餐
['', '無評', '自助餐 便當']
綠香Tea茶飲店(鹽水總店)
['', '無評', '飲料專賣店']
大排檔炭烤海鮮
['', '無評', '海鮮餐廳']
廣久港式燒臘
['', '無評', '粵菜 港式飲茶']
心悅便當
['', '無評', '自助餐 便當']
吳家羊肉 - 阿仁
['羊肉', '服務較差', '偏甜', '吳家', '清寧宮', '食物很香', '羊肉爐', '阿仁', '羊肉爐']
金科威補天鮮滴雞湯
['', '無評', '其他美食']
NUTS堅菓子手作工坊
['杏仁', '脆餅', '核桃', '倫子', '食物很香', '堅果', '甜點專賣']
ShabuSato日本涮涮鍋(台南店)
['價格較高昂', '涮涮鍋', '昆布', '梅花', '食物很香', '三越', '新光', '食物精緻漂亮', '日式']
義窩窩義式餐廳
['', '無評', '義式料理']
義起來創意廚坊
['', '無評', '義式料理']
歐派2號店
['', '無評', '甜點專賣']
泰冒煙南洋廚房
['', '無評', '泰式料理']
大豆豆嫩骨餐飲
['', '無評', '麵食點心']
鮮手作麵坊
['', '無評', '麵食點心']
讚不絕口小火鍋臭臭鍋專賣
['小火鍋', '趕時間', '找要', '豆皮', '演講', '其它中式料理']
KLG卡拉雞(長榮店)
['', '無評', '漢堡、炸雞']
小金鹽水雞
['必點脆', '圖下方', '科學', '百頁', '小金', '鹹水雞']
La france en silhouette山裡的

In [62]:
BigAnalyzeTable[600]['tags']

['蒼蠅', '天井', '閃電俠', '神農', '蛋糕', '複合式']

In [24]:
with open('D:/Data/JsonData/TainanFood/bigtable_1.0.json','w') as f:
    json.dump(BigAnalyzeTable,f)

In [25]:
# a=['食物精緻漂亮', '價格較高昂', '其他正面情緒']
# b=['食物美味', '價格較高昂', '環境好', '服務好', '食物精緻漂亮', '食物很香', '其他正面情緒', '重口味']
# c=['食物美味', '價格較高昂', '環境好', '服務好', '食物精緻漂亮', '食物很香', '其他正面情緒', '重口味']
# list(set(a+b+c))


In [26]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.0.json') as f:
    data=json.load(f)

In [27]:
data[1]

{'Ncomment': 20,
 'TF_IDF': {'一下': 0.001754501360184754,
  '一些': 0.0006625475617136504,
  '一份': 0.0007419207797875924,
  '一個': 0.0009011414526338026,
  '一口': 0.0007756626542362902,
  '一塊': 0.0016496494731986523,
  '一定': 0.0008443769623054691,
  '一層': 0.015352262498748439,
  '一杯': 0.001353826701220603,
  '一片': 0.0022637444136620805,
  '一直': 0.0007011741054788505,
  '一種': 0.0014483774876304326,
  '一起': 0.0009444982977212441,
  '一點': 0.0011415350190847838,
  '上面': 0.002314750148874417,
  '下': 0.0003777377902097156,
  '下午': 0.0009425675388220927,
  '下午茶': 0.0028464248333637645,
  '下去': 0.0011392210691898194,
  '不到': 0.0006020124265266177,
  '不膩': 0.0011451610859728118,
  '中': 0.000562689048181557,
  '中西區': 0.0006462439017702225,
  '中間': 0.001313807085719198,
  '之前': 0.0005377198023828408,
  '之旅': 0.0019174165007028313,
  '享受': 0.0011135675528753195,
  '人': 0.0005973312819124789,
  '今天': 0.0007198594499395265,
  '介紹': 0.0007171346059520806,
  '以上': 0.001373086522030121,
  '以為': 0.0008694200